
# OZON RecSys Baseline - Рекомендательная система для категории Apparel
Этот ноутбук содержит базовое решение для задачи предсказания следующей покупки пользователя в категории одежды, обуви и аксессуаров.

## Задача
- Предсказать топ-100 товаров для каждого пользователя из тестовой выборки
- Метрика оценки: NDCG@100
- Данные: ~38GB в формате parquet, 1.6B взаимодействий, 19M заказов


In [1]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from collections import defaultdict
import subprocess
import json
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Загрузка и анализ данных


In [2]:
def load_train_data():
    print("Загружаем тренировочные данные...")

    orders_files = glob.glob('/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_orders_data/*/*.parquet')
    tracker_files = glob.glob('/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_tracker_data/*/*.parquet')
    items_files = glob.glob('/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_items_data/*.parquet')
    categories_files = glob.glob('/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train_final_categories_tree/*.parquet')

    print(f"Найдено файлов заказов: {len(orders_files)}")
    print(f"Найдено файлов взаимодействий: {len(tracker_files)}")
    print(f"Найдено файлов товаров: {len(items_files)}")
    print(f"Найдено категорий: {len(categories_files)}")

    # --- Загружаем заказы ---
    orders_list = []
    for file_path in tqdm(orders_files, desc="Загрузка заказов"):
        df = pd.read_parquet(file_path)
        orders_list.append(df)
    orders_df = pd.concat(orders_list, ignore_index=True)
    print(f"Загружено заказов: {len(orders_df):,}")

    # --- Загружаем взаимодействия ---
    tracker_list = []
    for file_path in tqdm(tracker_files[:20], desc="Загрузка взаимодействий"):
        df = pd.read_parquet(file_path)
        tracker_list.append(df)
    tracker_df = pd.concat(tracker_list, ignore_index=True)
    print(f"Загружено взаимодействий: {len(tracker_df):,}")

    # --- Загружаем товары ---
    items_list = []
    for file_path in tqdm(items_files[:10], desc="Загрузка товаров"):
        df = pd.read_parquet(file_path)
        items_list.append(df)
    items_df = pd.concat(items_list, ignore_index=True)
    print(f"Загружено товаров: {len(items_df):,}")

    # --- Загружаем категории ---
    filter_ids = {7500, 7697, 17777}
    category_list = []

    for file_path in tqdm(categories_files, desc="Загрузка категорий"):
        df = pd.read_parquet(file_path)

        # Проверяем, что в 'ids' есть хотя бы один из filter_ids
        mask_ids = df['ids'].apply(lambda x: bool(set(x) & filter_ids))
        df = df[mask_ids]

        # Оставляем только строки с длиной списка ids == 5
        df = df[df['ids'].apply(lambda x: len(x) == 5)]

        if not df.empty:
            category_list.append(df)

    category_df = pd.concat(category_list, ignore_index=True) if category_list else pd.DataFrame(columns=df.columns)
    print(f"Загружено категорий: {len(category_df):,}")
    
    return orders_df, tracker_df, items_df, category_df


In [3]:
orders_df, tracker_df, items_df, category_df = load_train_data()

Загружаем тренировочные данные...
Найдено файлов заказов: 28
Найдено файлов взаимодействий: 298
Найдено файлов товаров: 100
Найдено категорий: 1


Загрузка заказов: 100%|██████████| 28/28 [00:00<00:00, 336.93it/s]


Загружено заказов: 1,475,216


Загрузка взаимодействий: 100%|██████████| 20/20 [00:00<00:00, 29.41it/s]


Загружено взаимодействий: 23,697,734


Загрузка товаров: 100%|██████████| 10/10 [00:09<00:00,  1.03it/s]


Загружено товаров: 644,800


Загрузка категорий: 100%|██████████| 1/1 [00:00<00:00, 23.43it/s]

Загружено категорий: 291


In [4]:
category_df

,catalogid,catalogpath,ids
0,7534,"[{'id': 7534, 'name': 'Дубленки и шубы', 'full...","[7534, 7528, 7501, 7500, -1]"
1,7529,"[{'id': 7529, 'name': 'Жилеты утепленные', 'fu...","[7529, 7528, 7501, 7500, -1]"
2,34590,"[{'id': 34590, 'name': 'Комплекты верхней одеж...","[34590, 7528, 7501, 7500, -1]"
3,7530,"[{'id': 7530, 'name': 'Куртки и пуховики', 'fu...","[7530, 7528, 7501, 7500, -1]"
4,7531,"[{'id': 7531, 'name': 'Пальто', 'fullName': 'П...","[7531, 7528, 7501, 7500, -1]"
...,...,...,...
286,7706,"[{'id': 7706, 'name': 'Дорожные сумки', 'fullN...","[7706, 7707, 8511, 7697, -1]"
287,37733,"[{'id': 37733, 'name': 'Комплектующие для чемо...","[37733, 7707, 8511, 7697, -1]"
288,11866,"[{'id': 11866, 'name': 'Ударопрочные кейсы', '...","[11866, 7707, 8511, 7697, -1]"
289,1657,"[{'id': 1657, 'name': 'Чемоданы', 'fullName': ...","[1657, 7707, 8511, 7697, -1]"


In [5]:
# items_df
items_df_apparel = items_df[items_df['catalogid'].isin(category_df['catalogid'])]
items_df_apparel

,item_id,itemname,attributes,fclip_embed,catalogid,variant_id,model_id
0,74396,Банты для девочек школьные желтые большие на р...,"[{'attribute_name': 'Type', 'attribute_value':...","[0.26911166, 0.796693, -0.3602935, 0.1430311, ...",17128,19226903,26519641
1,279348,Костюм спортивный Sonechka,"[{'attribute_name': 'Type', 'attribute_value':...","[0.37624118, -0.63848984, -0.40698087, 1.26255...",32812,145513553,36284667
4,1703201,Футболка классическая женская с принтом Лондо...,"[{'attribute_name': 'Type', 'attribute_value':...","[-0.5866448, 0.2311317, -1.6990204, 0.00280782...",36745,24915063,30233552
5,1709304,Комплект трусов бразильяна Inario Трусы женски...,"[{'attribute_name': 'Type', 'attribute_value':...","[0.2393944, -0.41456825, -0.35908598, 0.606085...",31306,156178758,9887542
6,1784801,Футболка Скуфаллон,"[{'attribute_name': 'Type', 'attribute_value':...","[0.4257005, -0.22235224, -1.0654651, 0.9875829...",7508,164633079,18717932
...,...,...,...,...,...,...,...
644794,338079700,"Комплект носков Unlimited, 6 пар","[{'attribute_name': 'Type', 'attribute_value':...","[-0.26794043, -1.5417383, -0.7986748, 0.283774...",7619,146449313,18102136
644795,338143463,Брюки H&M,"[{'attribute_name': 'Type', 'attribute_value':...","[-0.32066154, 0.4562481, -0.13283381, -0.36531...",36617,47507104,14793123
644796,338239916,Ботинки Shuzzi WildForce Girl,"[{'attribute_name': 'Name', 'attribute_value':...","[-0.1305742, -1.6921926, -1.0693192, 0.4007142...",36120,183173643,10383710
644798,338533496,Костюм для малышей Летний детский костюм с кор...,"[{'attribute_name': 'Type', 'attribute_value':...","[-0.57223, 0.6622674, -1.1859429, -0.12372005,...",7593,163582936,6240342


## 2. Exploratory Data Analysis (EDA)


In [6]:
orders_df

,item_id,user_id,created_timestamp,last_status,last_status_timestamp
0,79040838,7690,2025-07-09 21:18:46.100,delivered_orders,2025-07-10 17:31:37.000
1,334854442,15300,2025-07-09 09:54:36.130,canceled_orders,2025-07-09 10:08:24.270
2,249442281,18480,2025-07-09 00:42:00.400,canceled_orders,2025-07-13 20:40:29.837
3,164028972,18991,2025-07-09 11:36:58.673,proccesed_orders,2025-07-09 14:54:48.000
4,123996751,19821,2025-07-09 19:57:24.080,delivered_orders,2025-07-12 11:18:36.000
...,...,...,...,...,...
1475211,200947957,5022591,2025-07-14 12:59:44.250,proccesed_orders,2025-07-14 21:48:58.000
1475212,39592491,5025750,2025-07-14 17:10:27.720,proccesed_orders,2025-07-14 18:37:49.000
1475213,307398689,5029711,2025-07-14 19:59:36.780,proccesed_orders,2025-07-14 21:25:33.000
1475214,141792001,5036830,2025-07-14 07:27:29.476,proccesed_orders,2025-07-14 10:19:39.000


In [7]:
print("АНАЛИЗ ЗАКАЗОВ")
print("=" * 50)
print(f"Общее количество заказов: {len(orders_df):,}")
print(f"Уникальных пользователей: {orders_df['user_id'].nunique():,}")
print(f"Уникальных товаров: {orders_df['item_id'].nunique():,}")

# Если есть колонка created_date
if 'created_date' in orders_df.columns:
    print(f"Период данных: {orders_df['created_date'].min()} - {orders_df['created_date'].max()}")

print("\nРаспределение статусов заказов:")
status_counts = orders_df['last_status'].value_counts()

for status, count in status_counts.items():
    percentage = count / len(orders_df) * 100
    print(f"  {status}: {count:,} ({percentage:.1f}%)")

АНАЛИЗ ЗАКАЗОВ
Общее количество заказов: 1,475,216
Уникальных пользователей: 346,759
Уникальных товаров: 692,335

Распределение статусов заказов:
  proccesed_orders: 560,517 (38.0%)
  delivered_orders: 484,645 (32.9%)
  canceled_orders: 430,054 (29.2%)


In [8]:
print("\nАНАЛИЗ ВЗАИМОДЕЙСТВИЙ")
print("=" * 50)
print(f"Общее количество взаимодействий: {len(tracker_df):,}")
print(f"Уникальных пользователей: {tracker_df['user_id'].nunique():,}")
print(f"Уникальных товаров: {tracker_df['item_id'].nunique():,}")

print("\nРаспределение типов действий:")
action_counts = tracker_df['action_widget'].value_counts()
for action, count in action_counts.items():
    percentage = count / len(tracker_df) * 100
    print(f"  {action}: {count:,} ({percentage:.1f}%)")


АНАЛИЗ ВЗАИМОДЕЙСТВИЙ
Общее количество взаимодействий: 23,697,734
Уникальных пользователей: 558,964
Уникальных товаров: 2,124,173

Распределение типов действий:
  action_widget: 6,450,988 (27.2%)
  search: 6,156,974 (26.0%)
  recoms: 4,489,393 (18.9%)
  cart: 2,196,665 (9.3%)
  analogs: 1,992,035 (8.4%)
  favorites: 1,256,489 (5.3%)
  pdp: 1,155,190 (4.9%)


In [9]:
print("\nАНАЛИЗ ТОВАРОВ")
print("=" * 50)
print(f"Общее количество товаров: {len(items_df_apparel):,}")
print(f"Товары с эмбеддингами: {items_df_apparel['fclip_embed'].notna().sum():,}")

if 'attributes' in items_df_apparel.columns:
    print(f"Товары с атрибутами: {items_df_apparel['attributes'].notna().sum():,}")

print("\nТоп-10 категорий:")
top_categories = items_df_apparel['catalogid'].value_counts().head(10)
for cat_id, count in top_categories.items():
    print(f"  Категория {cat_id}: {count:,}")


АНАЛИЗ ТОВАРОВ
Общее количество товаров: 388,082
Товары с эмбеддингами: 388,082
Товары с атрибутами: 388,082

Топ-10 категорий:
  Категория 7559: 41,385
  Категория 7508: 26,727
  Категория 7530: 10,712
  Категория 17002: 10,548
  Категория 31307: 8,021
  Категория 36745: 7,890
  Категория 7506: 7,300
  Категория 31306: 6,698
  Категория 7540: 6,663
  Категория 7545: 6,010


## 3. Загрузка тестовых пользователей

In [11]:
def load_test_users():

    print("Загружаем тестовых пользователей...")
    
    test_files = glob.glob('/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_test_for_participants/test_for_participants/*.parquet')
    all_users = set()
    
    for file_path in tqdm(test_files, desc="Обработка тестовых файлов"):
        df = pd.read_parquet(file_path)
        all_users.update(df['user_id'].unique())
    
    print(f"Найдено уникальных тестовых пользователей: {len(all_users):,}")
    return list(all_users)

test_users = load_test_users()

Загружаем тестовых пользователей...


Обработка тестовых файлов: 100%|██████████| 1/1 [00:00<00:00, 24.46it/s]

Найдено уникальных тестовых пользователей: 470,347


In [27]:
test_users[:5]

[np.int32(1),
 np.int32(3145730),
 np.int32(3145731),
 np.int32(1048580),
 np.int32(4194310)]

## 4. Построение модели на основе популярности

In [20]:
def build_popularity_model(orders_df, tracker_df, items_df_apparel):
    print("Строим модель популярности...")

    # Получаем множество допустимых item_id
    valid_items = set(items_df_apparel['item_id'].unique())

    # Фильтруем заказы по статусу и допустимым товарам
    delivered_orders = orders_df[
        (orders_df['last_status'] == 'delivered_orders') &
        (orders_df['item_id'].isin(valid_items))
    ]
    print(f"Доставленных заказов (в категории): {len(delivered_orders):,}")

    item_popularity = delivered_orders['item_id'].value_counts().to_dict()
    print(f"Уникальных купленных товаров: {len(item_popularity):,}")

    # Фильтруем просмотры по допустимым товарам
    page_views = tracker_df[
        (tracker_df['action_type'] == 'page_view') &
        (tracker_df['item_id'].isin(valid_items))
    ]
    item_views = page_views['item_id'].value_counts().to_dict()
    print(f"Уникальных просмотренных товаров: {len(item_views):,}")

    # Считаем комбинированную популярность
    combined_popularity = defaultdict(float)
    for item_id, count in item_popularity.items():
        combined_popularity[item_id] += count * 3.0
    for item_id, count in item_views.items():
        combined_popularity[item_id] += count * 1.0

    # Сортируем и берем топ-200
    popular_items = sorted(combined_popularity.items(), key=lambda x: x[1], reverse=True)
    top_items = [item_id for item_id, score in popular_items[:200]]

    print(f"Отобрано топ-{len(top_items)} популярных товаров")

    # Топ-10 для вывода
    top_10_items = popular_items[:10]
    print("\nТоп-10 самых популярных товаров:")
    for i, (item_id, score) in enumerate(top_10_items, 1):
        print(f"  {i}. Товар {item_id}: {score:.1f} баллов")

    return top_items

# Пример вызова:
popular_items = build_popularity_model(orders_df, tracker_df, items_df_apparel)


Строим модель популярности...
Доставленных заказов (в категории): 32,055
Уникальных купленных товаров: 18,049
Уникальных просмотренных товаров: 113,165
Отобрано топ-200 популярных товаров

Топ-10 самых популярных товаров:
  1. Товар 102511397: 1209.0 баллов
  2. Товар 77696741: 970.0 баллов
  3. Товар 281974935: 869.0 баллов
  4. Товар 73912085: 832.0 баллов
  5. Товар 109966026: 828.0 баллов
  6. Товар 172560199: 718.0 баллов
  7. Товар 231770181: 708.0 баллов
  8. Товар 196151288: 658.0 баллов
  9. Товар 145849227: 653.0 баллов
  10. Товар 76813145: 646.0 баллов


In [21]:
def build_user_preferences(orders_df, tracker_df, test_users):

    print("Строим пользовательские предпочтения...")
    
    delivered_orders = orders_df[orders_df['last_status'] == 'delivered_orders']
    user_purchased_items = delivered_orders.groupby('user_id')['item_id'].apply(list).to_dict()
    print(f"Пользователей с покупками: {len(user_purchased_items):,}")
    
    user_viewed_items = tracker_df[tracker_df['action_type'] == 'page_view'].groupby('user_id')['item_id'].apply(list).to_dict()
    print(f"Пользователей с просмотрами: {len(user_viewed_items):,}")
    
    user_preferences = {}
    users_with_history = 0
    
    for user_id in tqdm(test_users, desc="Обработка предпочтений"):
        preferences = set()
        
        if user_id in user_purchased_items:
            preferences.update(user_purchased_items[user_id])
        
        if user_id in user_viewed_items:
            preferences.update(user_viewed_items[user_id][:50])
        
        user_preferences[user_id] = list(preferences)
        
        if len(preferences) > 0:
            users_with_history += 1
    
    print(f"Построены предпочтения для {len(user_preferences):,} пользователей")
    print(f"Пользователей с историей: {users_with_history:,} ({users_with_history/len(test_users)*100:.1f}%)")
    
    return user_preferences

user_preferences = build_user_preferences(orders_df, tracker_df, test_users)


Строим пользовательские предпочтения...
Пользователей с покупками: 217,648
Пользователей с просмотрами: 537,997


Обработка предпочтений: 100%|██████████| 470347/470347 [00:01<00:00, 348593.23it/s]


Построены предпочтения для 470,347 пользователей
Пользователей с историей: 381,411 (81.1%)


## 6. Генерация рекомендаций

In [22]:
def generate_recommendations(test_users, popular_items, user_preferences):

    print("Генерируем рекомендации...")
    
    recommendations = {}
    
    for user_id in tqdm(test_users, desc="Создание рекомендаций"):
        user_recs = []
        
        user_items = set(user_preferences.get(user_id, []))
        
        available_items = [item for item in popular_items if item not in user_items]
        
        if len(available_items) >= 100:
            user_recs = available_items[:100]
        else:
            user_recs = available_items + popular_items[:100-len(available_items)]
        
        if len(user_recs) < 100:
            remaining = 100 - len(user_recs)
            random_items = np.random.choice(popular_items, remaining, replace=True)
            user_recs.extend(random_items)
        
        recommendations[user_id] = user_recs[:100]
    
    print(f"Сгенерированы рекомендации для {len(recommendations):,} пользователей")
    
    rec_lengths = [len(recs) for recs in recommendations.values()]
    print(f"Средняя длина рекомендаций: {np.mean(rec_lengths):.1f}")
    print(f"Все рекомендации имеют 100 товаров: {all(l == 100 for l in rec_lengths)}")
    
    return recommendations

recommendations = generate_recommendations(test_users, popular_items, user_preferences)


Генерируем рекомендации...


Создание рекомендаций: 100%|██████████| 470347/470347 [00:04<00:00, 101303.65it/s]


Сгенерированы рекомендации для 470,347 пользователей
Средняя длина рекомендаций: 100.0
Все рекомендации имеют 100 товаров: True


## 7. Создание файла submission

In [24]:
def create_submission(recommendations, filename='/home/root6/python/e_cup/rec_system/result/baseline_submission.csv'):

    print(f"Создаем файл submission: {filename}")
    
    submission_data = []
    for user_id, items in tqdm(recommendations.items(), desc="Формирование submission"):
        submission_data.append({
            'user_id': user_id,
            'item_id_1 item_id_2 ... item_id_100': ' '.join(map(str, items))
        })
    
    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(filename, index=False)
    
    print(f"Сохранен submission с {len(submission_df):,} пользователями")
    print(f"Размер файла: {Path(filename).stat().st_size / 1024 / 1024:.1f} MB")
    
    return filename

submission_file = create_submission(recommendations)

Создаем файл submission: /home/root6/python/e_cup/rec_system/result/baseline_submission.csv


Формирование submission: 100%|██████████| 470347/470347 [00:02<00:00, 195345.65it/s]


Сохранен submission с 470,347 пользователями
Размер файла: 436.8 MB
